In [5]:
from pymongo import MongoClient
import pandas as pd
import pprint
import pymongo
import requests
import json

# import api_key
from api_keys import geoapify_key
# set Geoapify url
GEOCODE_URL = 'https://api.geoapify.com/v1/geocode/search'

In [6]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [7]:
print(mongo.list_database_names())

['North_Carolina', 'admin', 'config', 'epaDB', 'fruits_db', 'local', 'met_db', 'nc_housing_data', 'uk_food']


In [8]:
db = mongo['nc_housing_data']

In [9]:
# review the collections in our database
print(db.list_collection_names())

['cleaned_housing']


In [10]:
# assign the collection to a variable 
collection = db['cleaned_housing']

In [11]:
def check_if_empty():
    # Count the number of documents in the collection
    count = collection.count_documents({})
    
    if count == 0:
        print("The collection is empty.")
    else:
        print(f"The collection contains {count} documents.")

check_if_empty()

The collection contains 100 documents.


In [12]:
def print_db_contents():
    # Fetch all documents in the collection
    documents = collection.find()
    
    # Print each document
    print("Database Contents:")
    for document in documents:
        pprint.pprint(document)  # Use pprint to format the output

print_db_contents()

Database Contents:
{'1/31/2017': '43692.90943',
 '1/31/2018': '47056.90152',
 '1/31/2019': '48687.44439',
 '1/31/2020': '46742.59425',
 '1/31/2021': '54259.80225',
 '1/31/2022': '58347.20391',
 '1/31/2023': '55071.93402',
 '1/31/2024': '60016.99862',
 '10/31/2016': '44034.7277',
 '10/31/2017': '45564.05136',
 '10/31/2018': '48231.73542',
 '10/31/2019': '46729.96843',
 '10/31/2020': '52384.09886',
 '10/31/2021': '56330.50827',
 '10/31/2022': '57046.68324',
 '10/31/2023': '59216.94997',
 '11/30/2016': '43945.60839',
 '11/30/2017': '45991.36257',
 '11/30/2018': '48210.29508',
 '11/30/2019': '46566.80986',
 '11/30/2020': '53763.33715',
 '11/30/2021': '56843.91487',
 '11/30/2022': '56143.51023',
 '11/30/2023': '60041.11215',
 '12/31/2016': '43730.88182',
 '12/31/2017': '46460.90818',
 '12/31/2018': '48414.68881',
 '12/31/2019': '46703.99993',
 '12/31/2020': '54214.43103',
 '12/31/2021': '57365.18772',
 '12/31/2022': '55189.7509',
 '12/31/2023': '60139.20744',
 '2/28/2017': '43964.98124',
 '

In [13]:
def get_lat_lon(region_name, state_name):
    params = {
        'text': f'{region_name}, {state_name}, USA',  
        'apiKey': geoapify_key,
        'limit': 1
    }
    print(f"Requesting lat/lon for: {region_name}, {state_name}, USA") 
    
    try:
        response = requests.get(GEOCODE_URL, params=params)
        
        # Check the status code
        if response.status_code == 200:
            print("API request successful.")  # Debug print
            data = response.json()
            print("API Response:", data)  #  API response
            
            if data.get('features'):
                # Access the first feature in the results
                feature = data['features'][0]
                if 'geometry' in feature and 'coordinates' in feature['geometry']:
                    lon, lat = feature['geometry']['coordinates']
                    return lat, lon
                else:
                    print("No coordinates found in the response.")
            else:
                print("No features found in the response.")
            return None, None
        else:
            print(f"API request failed with status code {response.status_code}")
            print("Response content:", response.text)  # debugging
            return None, None
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return None, None

# Test the function
lat, lon = get_lat_lon('Wake County', 'NC')
print(f"Latitude: {lat}, Longitude: {lon}")

Requesting lat/lon for: Wake County, NC, USA
API request successful.
API Response: {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright'}, 'country': 'United States', 'country_code': 'us', 'state': 'North Carolina', 'county': 'Wake', 'lon': -78.6118311, 'lat': 35.7979355, 'state_code': 'NC', 'result_type': 'county', 'formatted': 'Wake, NC, United States of America', 'address_line1': 'Wake, NC', 'address_line2': 'United States of America', 'category': 'administrative', 'timezone': {'name': 'America/New_York', 'offset_STD': '-05:00', 'offset_STD_seconds': -18000, 'offset_DST': '-04:00', 'offset_DST_seconds': -14400, 'abbreviation_STD': 'EST', 'abbreviation_DST': 'EDT'}, 'plus_code': '8773Q9XQ+57', 'rank': {'importance': 0.7302463904976988, 'popularity': 3.652500535111446, 'confidence': 0.9

In [14]:
def get_lat_lon(region_name, state_name):
    params = {
        'text': f'{region_name}, {state_name}, USA', 
        'apiKey': geoapify_key,
        'limit': 1
    }
    print(f"Requesting lat/lon for: {region_name}, {state_name}, USA") 
    
    try:
        response = requests.get(GEOCODE_URL, params=params)
        
        # Check the status code
        if response.status_code == 200:
            print("API request successful.")  # Debug print
            data = response.json()
            print("API Response:", data) 
            
            if data.get('features'):
                feature = data['features'][0]
                if 'geometry' in feature and 'coordinates' in feature['geometry']:
                    lon, lat = feature['geometry']['coordinates']
                    return lat, lon
                else:
                    print("No coordinates found in the response.")
            else:
                print("No features found in the response.")
            return None, None
        else:
            print(f"API request failed with status code {response.status_code}")
            print("Response content:", response.text)  # Print the response context
            return None, None
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return None, None

def update_db():
    for document in collection.find():
        region_name = document.get('RegionName')  
        state_name = document.get('StateName')  
        if region_name and state_name:
            lat, lon = get_lat_lon(region_name, state_name)
            if lat and lon:
                collection.update_one(
                    {'_id': document['_id']},
                    {'$set': {'latitude': lat, 'longitude': lon}}
                )
                print(f'Updated {region_name} in {state_name} with lat: {lat}, lon: {lon}')
            else:
                print(f'Failed to get lat/lon for {region_name}, {state_name}')

update_db()

Requesting lat/lon for: Bertie County, NC, USA
API request successful.
API Response: {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright'}, 'country': 'United States', 'country_code': 'us', 'state': 'North Carolina', 'county': 'Bertie', 'lon': -76.9747965, 'lat': 36.0608505, 'state_code': 'NC', 'result_type': 'county', 'formatted': 'Bertie, NC, United States of America', 'address_line1': 'Bertie, NC', 'address_line2': 'United States of America', 'category': 'administrative', 'timezone': {'name': 'America/New_York', 'offset_STD': '-05:00', 'offset_STD_seconds': -18000, 'offset_DST': '-04:00', 'offset_DST_seconds': -14400, 'abbreviation_STD': 'EST', 'abbreviation_DST': 'EDT'}, 'plus_code': '8785326G+83', 'rank': {'importance': 0.7257311102599824, 'popularity': 1.3377464964220651, 'confide

In [16]:
def print_db_contents():
    # Fetch all documents in the collection
    documents = collection.find()
    
    # Print each document
    print("Database Contents:")
    for document in documents:
        pprint.pprint(document)

print_db_contents()

Database Contents:
{'1/31/2017': '43692.90943',
 '1/31/2018': '47056.90152',
 '1/31/2019': '48687.44439',
 '1/31/2020': '46742.59425',
 '1/31/2021': '54259.80225',
 '1/31/2022': '58347.20391',
 '1/31/2023': '55071.93402',
 '1/31/2024': '60016.99862',
 '10/31/2016': '44034.7277',
 '10/31/2017': '45564.05136',
 '10/31/2018': '48231.73542',
 '10/31/2019': '46729.96843',
 '10/31/2020': '52384.09886',
 '10/31/2021': '56330.50827',
 '10/31/2022': '57046.68324',
 '10/31/2023': '59216.94997',
 '11/30/2016': '43945.60839',
 '11/30/2017': '45991.36257',
 '11/30/2018': '48210.29508',
 '11/30/2019': '46566.80986',
 '11/30/2020': '53763.33715',
 '11/30/2021': '56843.91487',
 '11/30/2022': '56143.51023',
 '11/30/2023': '60041.11215',
 '12/31/2016': '43730.88182',
 '12/31/2017': '46460.90818',
 '12/31/2018': '48414.68881',
 '12/31/2019': '46703.99993',
 '12/31/2020': '54214.43103',
 '12/31/2021': '57365.18772',
 '12/31/2022': '55189.7509',
 '12/31/2023': '60139.20744',
 '2/28/2017': '43964.98124',
 '

In [17]:
def export_db_to_json():
    documents = list(collection.find())
    for doc in documents:
        doc['_id'] = str(doc['_id'])  # Convert ObjectId to string for JSON serialization

    with open('Resources/ten_year_data.json', 'w') as file:
        json.dump(documents, file, indent=4)
        
    print("Database export successful.")

export_db_to_json()

Database export successful.
